In [1]:
import os
import glob
from datetime import datetime, timezone
from dateutil import tz

import pandas as pd
import yfinance as yf
import requests
import numpy as np

ET = tz.gettz("American/New_York")


In [2]:
folder_path = "./Raw Historical Data/."

# get all tickers
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

tickers = []

for file in all_files:
    ticker = os.path.splitext(os.path.basename(file))[0]
    tickers.append(ticker)

print(tickers)

['AAPL', 'AMD', 'AMZN', 'BA', 'BAC', 'CAT', 'GOOGL', 'GS', 'HD', 'JNJ', 'JPM', 'KO', 'META', 'MSFT', 'NFLX', 'NVDA', 'PFE', 'PG', 'SPY', 'TSLA', 'XOM']


In [3]:
def to_utc_naive(ts):
    """accepts pandas/py datetime (tz-aware or not) and returns UTC naive"""
    if ts is None or pd.isna(ts):
        return None
    ts = pd.to_datetime(ts, utc=True)
    return ts.tz_convert("UTC").tz_localize(None)

In [4]:
def infer_amc_bmo_from_ts(utc_ts):
    """Classy if BMO/AMC from a UTC timestamp by converting to ET with DST."""
    if utc_ts is None or pd.isna(utc_ts):
        return "UNKNOWN"
    # Convert to ET
    et_ts = utc_ts.replace(tzinfo=timezone.utc).astimezone(ET)
    hhmm = et_ts.hour * 60 + et_ts.minute

    if hhmm < 570:
        return "BMO"
    elif hhmm >= 960:
        return "AMC"
    else:
        return "INTRADAY"
    

In [5]:
def surprise_pct(actual, est):
    if pd.isna(actual) or pd.isna(est) or est == 0:
        return np.nan
    return (actual - est) / abs(est)

In [6]:
def et_calendar_date(utc_ts):
    """Date the market would consider 'that earnings day in ET'"""
    if utc_ts is None or pd.isna(utc_ts):
        return pd.NaT
    et_ts = utc_ts.replace(tzinfo=timezone.utc).astimezone(ET)
    return pd.Timestamp(et_ts.date())

In [19]:
def fetch_yf_events(tickers, limit=24):
    records = []
    for t in tickers:
        try:
            df = yf.Ticker(t).get_earnings_dates(limit=limit)
            if df is None or df.empty:
                continue
            df = df.reset_index()
            df["ticker"] = t

            # Drop rows that are not Earnings
            if "Event Type" in df.columns:
                df = df[df["Event Type"].str.contains("Earnings", case=False, na=False)]

            df.rename(columns={
                "Earnings Date":"report_ts",
                "Reported EPS":"eps_actual",
                "EPS Estimate":"eps_consensus"
            }, inplace=True)
            # Normalize report_ts to UTC naive
            df["report_ts"] = pd.to_datetime(df["report_ts"], utc=True).dt.tz_convert("UTC").dt.tz_localize(None)

            # Apply amc_bmo classification
            df["amc_bmo"] = df["report_ts"].apply(infer_amc_bmo_from_ts)
            
            records.append(df[["ticker","report_ts", "amc_bmo", "eps_actual","eps_consensus"]])
        except Exception as e:
            print(f"Failed {t}: {e}")
    return pd.concat(records, ignore_index=True if records else pd.DataFrame())

        

In [12]:
def fetch_nasdaq(tickers):
    records = []
    headers = {"User-Agent": "Mozilla/5.0"}
    for t in tickers:
        try:
            url = f"https://api.nasdaq.com/api/company/{t}/earnings-surprise"
            r = requests.get(url, headers=headers, timeout=10)
            data = r.json()
            if not data or "data" not in data:
                continue
            table = data["data"].get("earningsSurpriseTable", {}).get("rows", [])
            for record in table:
                records.append({
                    "ticker": t,
                    "report_ts": record.get("dateReported"),  # usually "YYYY-MM-DD"
                    "eps_actual": record.get("eps"),
                    "eps_consensus": record.get("consensusForecast"),
                    "amc_bmo": "UNKNOWN"
                })
        except Exception as e:
            print(f"Failed {t}: {e}")
    
    return pd.DataFrame(records)

In [21]:
def merge_events(yf_df, nz_df):
    # Make sure both have the same column set
    common_cols = ["ticker","report_ts","amc_bmo","eps_actual","eps_consensus"]
    yf = yf_df[common_cols].copy()
    nz = nz_df[common_cols].copy()

    # Concat with yfinance first
    merged = pd.concat([yf, nz], ignore_index=True)

    # Drop duplicates, keep first
    merged = merged.sort_values(["ticker", "report_ts"], kind="mergesort")
    merged = merged.drop_duplicates(subset=["ticker", "report_ts"], keep="first")

    # sort for readability
    merged = merged.sort_values(["ticker", "report_ts"]).reset_index(drop=True)

    return merged

In [ ]:
test_df = fetch_nasdaq(tickers=tickers)
test_df.head()

Failed SPY: 'NoneType' object has no attribute 'get'


,ticker,report_ts,eps_actual,eps_consensus,amc_bmo
0,AAPL,7/31/2025,1.57,1.42,UNKNOWN
1,AAPL,5/1/2025,1.65,1.61,UNKNOWN
2,AAPL,1/30/2025,2.40,2.36,UNKNOWN
3,AAPL,10/31/2024,1.64,1.49,UNKNOWN
4,AMD,8/5/2025,0.27,0.28,UNKNOWN


In [28]:
test2_df = fetch_yf_events(tickers=tickers, limit=40)
test2_df.head()

,ticker,report_ts,amc_bmo,eps_actual,eps_consensus
0,AAPL,2025-05-01 20:30:00,AMC,1.65,1.63
1,AAPL,2025-01-30 21:31:00,AMC,2.40,2.35
2,AAPL,2024-10-31 20:31:00,AMC,1.64,1.60
3,AAPL,2024-08-01 20:30:00,AMC,1.40,1.35
4,AAPL,2024-05-02 20:31:00,AMC,1.53,1.50


In [29]:
test3 = merge_events(test2_df, test_df)
test3.count

<bound method DataFrame.count of     ticker            report_ts   amc_bmo  eps_actual eps_consensus
0     AAPL  2015-10-27 20:30:00       AMC        0.49          0.47
1     AAPL  2016-01-26 21:28:00       AMC        0.82          0.81
2     AAPL  2016-04-26 20:30:00       AMC        0.48           0.5
3     AAPL  2016-07-26 20:30:00       AMC        0.36          0.35
4     AAPL  2016-10-25 20:30:00       AMC        0.42          0.41
..     ...                  ...       ...         ...           ...
850    XOM  2025-05-02 10:30:00  INTRADAY        1.76          1.73
851    XOM            1/31/2025   UNKNOWN        1.67          1.55
852    XOM            11/1/2024   UNKNOWN        1.92          1.91
853    XOM             5/2/2025   UNKNOWN        1.76          1.74
854    XOM             8/1/2025   UNKNOWN        1.64          1.49

[855 rows x 5 columns]>

In [23]:
# Yahoo EPS

ticker = "AAPL"
# def get_earnings_data(ticker):
    # yahoo EPS
tkr = yf.Ticker(ticker)
eps_df = tkr.get_earnings_dates(limit=200).reset_index()
eps_df.rename(columns={
    "Earnings Date": "report_ts",
    "Reported EPS": "eps_actual",
    "EPS Estimate": "eps_consensus"
}, inplace=True)

print(eps_df.head())

# Nasdaq EPS + Rev
url = f"https://api.nasdaq.com/api/company/{ticker}/earnings-surprise"
headers = {"User-Agent": "Mozilla/5.0"}
r = requests.get(url, headers=headers).json()
rows = (r or {}).get("data", {}).get("earningsSurpriseTable", {}).get("rows", []) or []
rev_df = pd.DataFrame(rows)
if not rev_df.empty:
    rev_df = rev_df.rename(columns={
        "reportDate": "report_ts",
        "epsActual": "eps_actual",
        "epsConsensus": "eps_consensus"
    })
    # rev_df["report_ts"] = pd.to_datetime(rev_df["report_ts"], errors="coerce")
    # rev_df["report_date"] = rev_df["report_ts"].dt.date
else:
    rev_df = pd.DataFrame(columns=["report_date","report_ts","rev_actual","rev_consensus"])
print(rev_df.head())

                  report_ts  eps_consensus  eps_actual  Surprise(%) Event Type
0 2025-05-01 16:30:00-04:00           1.63        1.65         1.41   Earnings
1 2025-02-27 12:00:00-05:00            NaN         NaN          NaN    Meeting
2 2025-01-30 16:31:00-05:00           2.35        2.40         2.15   Earnings
3 2024-10-31 16:31:00-04:00           1.60        1.64         2.35   Earnings
4 2024-08-01 16:30:00-04:00           1.35        1.40         3.99   Earnings
  fiscalQtrEnd dateReported   eps consensusForecast percentageSurprise
0     Jun 2025    7/31/2025  1.57              1.42              10.56
1     Mar 2025     5/1/2025  1.65              1.61               2.48
2     Dec 2024    1/30/2025  2.40              2.36               1.69
3     Sep 2024   10/31/2024  1.64              1.49              10.07
